### H2O.ai Explain Results 

This notebook trains an H2O AutoML Model on a subset of the data and generates automated explanations for the model.

In [ ]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

h2o.init()

# Import the data keeping only the final transaction for each customer
df = pd.read_csv("../input/amex-default-prediction/train_data.csv", nrows=500000)
df = df.sort_values(['customer_ID', 'S_2'])
df = df.drop_duplicates(subset='customer_ID', keep='last')
train_labels =  pd.read_csv("../input/amex-default-prediction/train_labels.csv") 

# Add labels 
df = df.merge(train_labels, on = 'customer_ID', how='left')

# Reponse column
y = "target"

# Features to be included
features = list(df.columns)
features.remove('customer_ID')
features.remove('S_2')
features.remove(y)

# Convert to H2O frame
df = h2o.H2OFrame(df)


In [ ]:
# Split into train & test
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

# Change the target to a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
# Run H2O.ai AutoML for 30 minutes
aml = H2OAutoML(max_runtime_secs=1800, seed=1)
aml.train(y=y, x=features, training_frame=train)

In [ ]:
# Explain leader model & compare with all AutoML models
exa = aml.explain(test, top_n_features=20)

In [ ]:
# Explain a single H2O model (e.g. leader model from AutoML)
# exm = aml.leader.explain(test, top_n_features=10)

In [ ]:
# h2o.cluster().shutdown(prompt=False) 